Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Your dataset should be sufficiently large for a neural network to perform well (samples should really be in the thousands here) and try to pick something that takes advantage of neural networks’ ability to have both feature extraction and supervised capabilities, so don’t pick something with an easy to consume list of features already generated for you (though neural networks can still be useful in those contexts).

Note that if you want to use an unprocessed image dataset, scikit-image is a useful package for converting to importable numerics.

In [15]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: ignored

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [17]:
!ls

'6.6.6 Challange, tensorflow and keras.ipynb'


In [0]:
import tensorflow as tf
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [20]:
#import first part of data
wine1 = pd.read_csv('winemag-data-first.csv')
#import second part of data
wine2 = pd.read_csv('winemag-data-second.csv')
#combine wine data
wine_merge = wine1.append(wine2, ignore_index=True)
#print head
wine_merge.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,NaN,NaN,NaN,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,NaN,NaN,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,NaN,NaN,NaN,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,NaN,NaN,NaN,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,NaN,NaN,NaN,Provence red blend,Domaine de la Bégude


In [21]:
#get column names
wine_merge.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [22]:
#drop irrelevant colums
wine = wine_merge.drop(['Unnamed: 0','description','region_2','taster_name','taster_twitter_handle','title'],axis=1)
#drop null
wine = wine.dropna()
#print info
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147459 entries, 0 to 280900
Data columns (total 8 columns):
country        147459 non-null object
designation    147459 non-null object
points         147459 non-null int64
price          147459 non-null float64
province       147459 non-null object
region_1       147459 non-null object
variety        147459 non-null object
winery         147459 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 10.1+ MB


In [23]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
col = ['country','variety','winery','region_1','province','designation']
wine[col] = wine[col].apply(lambda x: le.fit_transform(x))
wine.head()

,country,designation,points,price,province,region_1,variety,winery
0,6,20879,96,235.0,8,715,47,5608
1,5,5276,96,110.0,39,1042,437,1016
2,6,30259,96,90.0,8,513,372,6988
3,6,26636,96,65.0,43,1190,322,8603
4,3,17075,95,66.0,47,60,335,4104


In [0]:
from sklearn.preprocessing import OneHotEncoder

#ohe = OneHotEncoder()
#ohe.fit(wine['country'].values.reshape(1,-1))
#ohe.transform(wine['country'].values.reshape(1,-1)).toarray()[0]

In [0]:
from keras.utils import to_categorical



# Specify the data 
X=wine.drop(['country'],axis=1)

# Specify the target labels and flatten the array 
y=to_categorical(wine['country'])


# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1337, stratify=y)

In [26]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(100, activation='relu', input_dim= X_train.shape[1] ))

# Add a hidden layer 
model.add(Dense(100, activation='relu'))

# Add an output layer 
model.add(Dense(7, activation='softmax'))

model.summary()

W0618 03:50:38.186343 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0618 03:50:38.248881 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0618 03:50:38.260551 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 707       
Total params: 11,607
Trainable params: 11,607
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=10, batch_size=512, verbose=1,validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0618 03:50:42.168314 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0618 03:50:42.183020 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0618 03:50:42.332171 140019014723456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0618 03:50:42.411559 140019014723456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Train on 98797 samples, validate on 48662 samples
Epoch 1/10
98797/98797 [==============================] - 2s 16us/step - loss: 7.8079 - acc: 0.5155 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 2/10
98797/98797 [==============================] - 1s 9us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 3/10
98797/98797 [==============================] - 1s 9us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 4/10
98797/98797 [==============================] - 1s 9us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 5/10
98797/98797 [==============================] - 1s 9us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 6/10
98797/98797 [==============================] - 1s 10us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 7/10
98797/98797 [==============================] - 1s 9us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5

ATTEMPT 2, extra layers

In [28]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(100, activation='relu', input_dim= X_train.shape[1] ))

# Add a hidden layer 
model.add(Dense(100, activation='relu'))

# Add a hidden layer 
model.add(Dense(100, activation='relu'))

# Add a hidden layer 
model.add(Dense(100, activation='relu'))

# Add an output layer 
model.add(Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 707       
Total params: 31,807
Trainable params: 31,807
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=10, batch_size=512, verbose=1,validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 98797 samples, validate on 48662 samples
Epoch 1/10
98797/98797 [==============================] - 2s 18us/step - loss: 13.5587 - acc: 0.1588 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 2/10
98797/98797 [==============================] - 2s 16us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 3/10
98797/98797 [==============================] - 2s 16us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 4/10
98797/98797 [==============================] - 1s 15us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 5/10
98797/98797 [==============================] - 1s 15us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 6/10
98797/98797 [==============================] - 2s 15us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_acc: 0.5196
Epoch 7/10
98797/98797 [==============================] - 1s 15us/step - loss: 7.7433 - acc: 0.5196 - val_loss: 7.7431 - val_ac

Attempt 3, more nodes

In [30]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(500, activation='relu', input_dim= X_train.shape[1] ))

# Add a hidden layer 
model.add(Dense(500, activation='relu'))

# Add an output layer 
model.add(Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 500)               4000      
_________________________________________________________________
dense_10 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 3507      
Total params: 258,007
Trainable params: 258,007
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=10, batch_size=512, verbose=1,validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 98797 samples, validate on 48662 samples
Epoch 1/10
98797/98797 [==============================] - 5s 54us/step - loss: 13.7382 - acc: 0.1476 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 2/10
98797/98797 [==============================] - 5s 51us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 3/10
98797/98797 [==============================] - 5s 51us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 4/10
98797/98797 [==============================] - 5s 51us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 5/10
98797/98797 [==============================] - 5s 52us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 6/10
98797/98797 [==============================] - 5s 52us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.7379 - val_acc: 0.1477
Epoch 7/10
98797/98797 [==============================] - 5s 52us/step - loss: 13.7377 - acc: 0.1477 - val_loss: 13.

Attempt 4, more nodes and more layers

In [32]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(500, activation='relu', input_dim= X_train.shape[1] ))

# Add a hidden layer 
model.add(Dense(500, activation='relu'))

# Add a hidden layer 
model.add(Dense(500, activation='relu'))

# Add a hidden layer 
model.add(Dense(500, activation='relu'))

# Add an output layer 
model.add(Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 500)               4000      
_________________________________________________________________
dense_13 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_14 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_15 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_16 (Dense)             (None, 7)                 3507      
Total params: 759,007
Trainable params: 759,007
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=10, batch_size=512, verbose=1,validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 98797 samples, validate on 48662 samples
Epoch 1/10
98797/98797 [==============================] - 13s 132us/step - loss: 15.3838 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 2/10
98797/98797 [==============================] - 13s 132us/step - loss: 15.3836 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 3/10
98797/98797 [==============================] - 13s 133us/step - loss: 15.3836 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 4/10
98797/98797 [==============================] - 13s 133us/step - loss: 15.3836 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 5/10
98797/98797 [==============================] - 13s 133us/step - loss: 15.3836 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 6/10
98797/98797 [==============================] - 13s 132us/step - loss: 15.3836 - acc: 0.0456 - val_loss: 15.3834 - val_acc: 0.0456
Epoch 7/10
98797/98797 [==============================] - 13s 133us/step - loss: 15.3836 - acc: 0.0456 -

Attempt 5, different function

In [34]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(100, activation='sigmoid', input_dim= X_train.shape[1] ))

# Add a hidden layer 
model.add(Dense(100, activation='sigmoid'))

# Add a hidden layer 
model.add(Dense(100, activation='sigmoid'))

# Add a hidden layer 
model.add(Dense(100, activation='sigmoid'))

# Add an output layer 
model.add(Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 100)               800       
_________________________________________________________________
dense_18 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_21 (Dense)             (None, 7)                 707       
Total params: 31,807
Trainable params: 31,807
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=10, batch_size=512, verbose=1,validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 98797 samples, validate on 48662 samples
Epoch 1/10
98797/98797 [==============================] - 2s 20us/step - loss: 1.4060 - acc: 0.5143 - val_loss: 1.3985 - val_acc: 0.5196
Epoch 2/10
98797/98797 [==============================] - 1s 15us/step - loss: 1.3827 - acc: 0.5196 - val_loss: 1.3634 - val_acc: 0.5199
Epoch 3/10
98797/98797 [==============================] - 1s 15us/step - loss: 1.3547 - acc: 0.5212 - val_loss: 1.3391 - val_acc: 0.5232
Epoch 4/10
98797/98797 [==============================] - 1s 14us/step - loss: 1.3415 - acc: 0.5229 - val_loss: 1.3503 - val_acc: 0.5173
Epoch 5/10
98797/98797 [==============================] - 1s 15us/step - loss: 1.3380 - acc: 0.5245 - val_loss: 1.3413 - val_acc: 0.5273
Epoch 6/10
98797/98797 [==============================] - 1s 15us/step - loss: 1.3389 - acc: 0.5263 - val_loss: 1.3338 - val_acc: 0.5299
Epoch 7/10
98797/98797 [==============================] - 1s 14us/step - loss: 1.3332 - acc: 0.5285 - val_loss: 1.3398 - val_acc